In [ ]:
import glob
from pathlib import Path

import pandas as pd
import xarray as xr

from blooms_ml.utils import (
    sample_stations,
    get_from_dia,
    get_from_avg,
)

In [ ]:
files_dia = sorted(glob.glob(
    f"{Path.home()}/fram_shmiak/ROHO800_hindcast_2007_2019_v2bu/roho800_v2bu_dia/*dia*.nc"
))[:20]
files_avg = sorted(glob.glob(
    f"{Path.home()}/fram_shmiak/ROHO800_hindcast_2007_2019_v2bu/roho800_v2bu_avg/*avg*.nc"
))[:20]

In [ ]:
ds_dia = xr.open_mfdataset(files_dia)
ds_avg = xr.open_mfdataset(files_avg)
stations, st_labels, xis, etas = sample_stations(ds_dia, 100)

ddf_dia = get_from_dia(ds_dia, xis, etas)
ddf = get_from_avg(ds_avg, xis, etas)

ddf['light_PAR0'] = ddf_dia['light_PAR0']
ddf['P1_netPI'] = ddf_dia['P1_netPI']
df = ddf.compute()

In [ ]:
def normalize_series(row: pd.Series):
    return ((row - row.mean()) / row.std()).round(2).astype('float32')

def append_rho_profiles(df_station, nlayers: int = 25):
    df_station = df_station.reset_index(drop=True)
    rho = df_station.pivot(index='ocean_time', columns='s_rho', values='rho')
    new_columns = [str(i) for i in range(1, len(rho.columns)+1)]
    rho.rename(columns=dict(zip(rho.columns[:], new_columns)), inplace=True)
    rho = rho.apply(normalize_series, axis=1)
    rho = rho.loc[rho.index.repeat(nlayers)]
    rho = rho.rename_axis(None, axis=1)
    rho = rho.reset_index()
    return pd.concat([df_station, rho.iloc[:, 1:]], axis=1)

In [ ]:
df = df.groupby('station').apply(append_rho_profiles)

In [ ]:
df = df[df['s_rho'] > -0.3]  # surface
df = df.reset_index(drop=True)
df.iloc[:, 3:11] = df.iloc[:, 3:11].apply(normalize_series, axis=0)

In [ ]:
df

In [ ]:
df.to_parquet(f"{Path.home()}/data_ROHO/blooms-ml-stations.parquet")